# Redes neuronales: CNN


## 1.- Descripción

Las Redes Neuronales Convolucionales (CNN, por sus siglas en inglés) son un tipo especial de red neuronal que se ha demostrado ser extremadamente eficaz para el procesamiento de datos estructurados en forma de grillas, como imágenes, series temporales y datos de audio. Este modelo se inspira en el sistema visual de los seres humanos, lo que le permite identificar patrones jerárquicos de manera eficiente, desde características simples (como bordes y texturas) hasta patrones más complejos (como formas o objetos completos).

En este proyecto, el objetivo es aplicar una red neuronal convolucional para un problema de clasificación binaria. Utilizaremos las capacidades de las CNN para extraer características relevantes del conjunto de datos y realizar una clasificación precisa, optimizando tanto las métricas de precisión como de recall, especialmente cuando se trata de un conjunto de datos desbalanceado. Se busca que el modelo no solo sea preciso, sino que también sea capaz de manejar los datos desbalanceados de manera efectiva, garantizando una clasificación más robusta en ambas clases.

Este enfoque de modelado con CNN puede ser útil para diversas aplicaciones, no solo en visión por computadora, sino también en dominios como la clasificación de textos, predicción de series temporales, y en cualquier escenario donde las relaciones espaciales y jerárquicas sean importantes.

## 2.- Implementación


In [74]:
# Importar las librerías necesarias
#Están incluidas librerías que fueron usadas como prueba
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import Adam

In [75]:
# Cargar los datos
df = pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\processed\new\train_balanced_processed.csv")
df_test= pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\processed\new\test_nolabel_processed.csv")


In [76]:
# Balancear el dataset con undersampling
df_majority = df[df["Accept"] == 1]
df_minority = df[df["Accept"] == 0]
df_majority_downsampled = resample(df_majority, 
                                   replace=False,
                                   n_samples=len(df_minority),
                                   random_state=42)

In [77]:
df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [78]:
# Separar X e y
X = df_balanced.drop('Accept', axis=1)
y = df_balanced['Accept']

In [79]:
df_test_features = df_test.drop(columns=['id'], errors='ignore') 

In [80]:
train_columns = X.columns

In [81]:
# Codificar texto si hay columnas categóricas
le = LabelEncoder()
categorical_columns = X.select_dtypes(include=['object']).columns
for col in categorical_columns:
    X[col] = le.fit_transform(X[col].astype(str))


In [82]:
categorical_columns_test = df_test.select_dtypes(include=['object']).columns

categorical_columns_test = [col for col in categorical_columns_test if col != 'id']
for col in categorical_columns_test:
    if col in categorical_columns:  # Solo transformar las columnas que están en df_train
        # Reemplazar las categorías desconocidas con 'Unknown' si es necesario
        if 'Unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'Unknown')
        
        # Reemplazar las categorías desconocidas con 'Unknown' y aplica la transformación
        df_test[col] = df_test[col].astype(str).apply(lambda x: x if x in le.classes_ else 'Unknown')
        df_test[col] = le.transform(df_test[col])

In [83]:
categorical_columns_test = df_test_features.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_columns_test:
    df_test_features[col] = le.fit_transform(df_test_features[col].astype(str))


In [84]:
# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
df_test_imputed = imputer.fit_transform(df_test_features)

In [85]:
# Escalar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
df_test_scaled = scaler.fit_transform(df_test_imputed)

In [86]:
# Dividir en train/test

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)



In [87]:
print(df_test.head())

            id  NewExist  UrbanRural  RevLineCr  LowDoc  BankStateInOhio  \
0  6b7ce9ec161         1           2          1       0                0   
1  96123015731         1           1          1       0                1   
2  b2c5181ac5b         1           2          1       0                0   
3  e6cb54a9e6a         2           1          0       0                0   
4  8eddf83466e         1           0          0       0                1   

   ApprovalDateMonth  ApprovalFYGrouped  NoEmpGrouped  CreateJobBinary  \
0                 10               2005             0                0   
1                  4               2000             1                1   
2                  1               2003             1                0   
3                  7               2004             1                1   
4                 11               1991             1                1   

   RetainedJobBinary  IsFranchise  DisbursementGrossGrouped  
0                  1            0   

In [88]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de X_train: (4681, 12)
Dimensiones de df_test_scaled: (3284, 12)


In [89]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de X_train: (4681, 12)
Dimensiones de df_test_scaled: (3284, 12)


## 3.- Evaluación y optimización


In [91]:
# Crear y entrenar el modelo MLP
#A lo largo del entrenamiento se fueron modificando las diferentes características del modelo para buscar optimizarlo.
model = Sequential([
    Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], 1)),  # Capa convolucional
    MaxPooling1D(pool_size=2),  # Capa de MaxPooling
    Dropout(0.3),  # Dropout para evitar sobreajuste
    Flatten(),  # Aplanar las características
    Dense(128, activation='relu'),  # Capa densa con activación ReLU
    Dropout(0.3),  # Dropout para evitar sobreajuste
    Dense(1, activation='sigmoid')  # Capa de salida con activación sigmoide para clasificación binaria
])

# Compilar el modelo con la función de pérdida binary_crossentropy
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [92]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 8, 64)             384       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 4, 64)            0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 4, 64)             0         
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                      

In [93]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/30
117/117 [==============================] - 3s 16ms/step - loss: 0.6661 - accuracy: 0.5967 - val_loss: 0.6409 - val_accuracy: 0.6350
Epoch 2/30
117/117 [==============================] - 1s 12ms/step - loss: 0.6517 - accuracy: 0.6213 - val_loss: 0.6323 - val_accuracy: 0.6446
Epoch 3/30
117/117 [==============================] - 2s 13ms/step - loss: 0.6424 - accuracy: 0.6263 - val_loss: 0.6317 - val_accuracy: 0.6286
Epoch 4/30
117/117 [==============================] - 1s 11ms/step - loss: 0.6401 - accuracy: 0.6207 - val_loss: 0.6328 - val_accuracy: 0.6222
Epoch 5/30
117/117 [==============================] - 1s 9ms/step - loss: 0.6365 - accuracy: 0.6341 - val_loss: 0.6314 - val_accuracy: 0.6329
Epoch 6/30
117/117 [==============================] - 1s 10ms/step - loss: 0.6363 - accuracy: 0.6319 - val_loss: 0.6288 - val_accuracy: 0.6329
Epoch 7/30
117/117 [==============================] - 1s 10ms/step - loss: 0.6314 - accuracy: 0.6381 - val_loss: 0.6274 - val_accuracy: 0.6339


In [94]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de X_train: (4681, 12)
Dimensiones de df_test_scaled: (3284, 12)


In [95]:
# Predecir probabilidades
y_pred= model.predict(X_test)


37/37 [==============================] - 0s 3ms/step


In [96]:
y_pred_prob = model.predict(X_test)

y_pred = (y_pred_prob > 0.4).astype(int)

37/37 [==============================] - 0s 2ms/step


In [97]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)


Dimensiones de X_train: (4681, 12)
Dimensiones de df_test_scaled: (3284, 12)


In [98]:
# Hacer predicciones
y_test_pred = model.predict(df_test_scaled)
y_test_bin = (y_test_pred > 0.5).astype(int)


103/103 [==============================] - 0s 3ms/step


In [99]:
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de df_test_scaled: (3284, 12)


In [100]:
# Evaluar
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[311 278]
 [143 439]]
              precision    recall  f1-score   support

           0       0.69      0.53      0.60       589
           1       0.61      0.75      0.68       582

    accuracy                           0.64      1171
   macro avg       0.65      0.64      0.64      1171
weighted avg       0.65      0.64      0.64      1171



In [101]:
# Crear DataFrame con las predicciones (id y Accept)
df_result = df_test[['id']].copy()  # Agregar 'id' del conjunto de test
df_result['ACCEPT'] = y_test_bin
print(df_result.head())

            id  ACCEPT
0  6b7ce9ec161       0
1  96123015731       1
2  b2c5181ac5b       0
3  e6cb54a9e6a       0
4  8eddf83466e       1


In [102]:
# Guardar el resultado en un archivo CSV
df_result.to_csv("SUBMMIT_CNN.csv", index=False)

print("¡Predicciones completas y archivo guardado como 'resultado_test.csv'!")


¡Predicciones completas y archivo guardado como 'resultado_test.csv'!


In [103]:
df_result.shape

(3284, 2)

In [104]:
# Verificar las dimensiones de df_test_scaled y y_test_pred
print(f"Dimensiones de df_test_scaled: {df_test_scaled.shape}")
print(f"Dimensiones de y_test_pred: {y_test_pred.shape}")


Dimensiones de df_test_scaled: (3284, 12)
Dimensiones de y_test_pred: (3284, 1)


## 4.- Conclusión

El modelo basado en redes neuronales convolucionales (CNN) alcanzó una exactitud global del 63% en la predicción de decisiones sobre préstamos bancarios, lo que indica un desempeño moderado en términos generales. El modelo mostró un comportamiento desigual entre las clases: tuvo un mejor desempeño al identificar los préstamos aprobados (clase 1), con un recall del 79%, lo que sugiere una alta capacidad para detectar correctamente a los solicitantes aceptados.

Sin embargo, la capacidad del modelo para identificar préstamos rechazados (clase 0) fue más limitada, con un recall del 47%, lo que indica una proporción considerable de falsos negativos en esta clase. Este desequilibrio se refleja también en las métricas de f1-score: 0.68 para la clase aprobada y 0.56 para la clase rechazada.

En resumen, aunque el modelo CNN ofrece un rendimiento aceptable y es particularmente eficaz para detectar casos positivos (préstamos aprobados), sería recomendable implementar estrategias para mejorar la detección de préstamos rechazados. Entre las posibles acciones se incluyen el ajuste de umbrales de decisión, el uso de técnicas de balanceo de clases o la evaluación de arquitecturas más adecuadas para datos tabulares.